In [3]:
import pandas as pd
import numpy as np
from model import get_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ReLU
from tensorflow.keras.activations import tanh
from tensorflow.keras.layers import BatchNormalization
import keras

In [4]:
rain = pd.read_csv("./Data/Rainfall.csv")
rain.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec,Anomaly
0,1901,34.7,38.6,17.8,38.9,50.6,113.2,241.4,271.6,124.7,52.4,38.7,8.2,1030.8,73.2,107.3,751.0,99.3,-122.62
1,1902,7.4,4.2,19.0,44.1,48.8,111.7,284.9,201.0,200.2,62.5,29.4,25.2,1038.4,11.6,111.9,797.8,117.2,-75.82
2,1903,16.7,8.0,31.1,17.1,59.5,120.3,293.2,274.0,198.1,119.5,40.3,18.0,1195.9,24.7,107.7,885.6,177.8,11.98
3,1904,14.9,9.7,31.4,33.7,73.8,165.5,260.3,207.7,130.8,69.8,11.2,16.4,1025.1,24.5,138.8,764.3,97.4,-109.32
4,1905,24.7,20.3,41.8,33.8,55.8,93.7,253.0,201.7,178.1,54.9,9.6,10.1,977.5,45.0,131.4,726.4,74.7,-147.22


In [5]:
predictor = pd.read_csv("./Data/10_para_Data.csv")
predictor.head()

,Year,NINO_34,EastAsiaSurfacePressure,NorthAtlantaicSurfcePressure,NorthAtlanticOcean_SST,EQ_WIN,AO_oct,PDO_Jun,North_central_Pacific_zonal_Wind,SOI_Jun,SEI_Ocean_SST,ISMR
0,1950,0.365675,-0.386439,-2.546593,0.572334,-0.195580,-0.212910,-2.293911,-9.595640,2.0,0.039908,876.9
1,1951,1.090162,-0.157037,-1.704145,-0.359634,-1.003503,-0.437170,-1.466423,-8.201849,0.2,-0.908400,738.8
2,1952,-0.150832,1.845762,-2.880367,0.585410,-1.739785,-0.194460,-1.982165,-8.985010,0.7,-0.672092,792.9
3,1953,0.347131,1.401160,-2.243482,-0.393405,-1.669605,0.512570,-0.226286,-9.018811,0.1,-0.672323,922.9
4,1954,-1.080947,1.755364,-0.979263,0.021049,0.699660,0.099053,0.475692,-8.474271,0.1,0.012831,885.3


In [6]:
predictor.iloc[0]

Year                                1950.000000
NINO_34                                0.365675
EastAsiaSurfacePressure               -0.386439
NorthAtlantaicSurfcePressure          -2.546593
NorthAtlanticOcean_SST                 0.572334
EQ_WIN                                -0.195580
AO_oct                                -0.212910
PDO_Jun                               -2.293911
North_central_Pacific_zonal_Wind      -9.595640
SOI_Jun                                2.000000
SEI_Ocean_SST                          0.039908
ISMR                                 876.900000
Name: 0, dtype: float64

In [7]:
import glob

slp_path = "./Data/slp/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)
  
slp_months.sort()
print(slp_months)

['./Data/slp\\01_slp.csv', './Data/slp\\02_slp.csv', './Data/slp\\03_slp.csv', './Data/slp\\04_slp.csv', './Data/slp\\05_slp.csv', './Data/slp\\06_slp.csv', './Data/slp\\07_slp.csv', './Data/slp\\08_slp.csv', './Data/slp\\09_slp.csv', './Data/slp\\10_slp.csv', './Data/slp\\11_slp.csv', './Data/slp\\12_slp.csv']


In [8]:
#Here i am just storing rainfall data of Jun to sep from 1949 to 2014
# index 47= 1948,
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:89]))
print(lpa)
rain_fall_data = list(rain['Jun-Sep'][47:2020])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)
  print(temp)

878.0689655172414
110.74300973923971
108.34000942507068
103.12401822180333
86.94077913917687
94.40032987747408
109.83191957273013
104.92342130065975
107.50863964813068
111.98437009110901
96.92860508953818
112.89546025761858
119.31864593151116
104.19454916745208
119.80835689601005
103.2492931196984
103.43151115300032
113.54461200125667
80.68842287150487
89.71960414703109
100.69824065347157
91.3026233113415
101.81432610744581
112.41713792020107
103.72761545711593
76.7935124096764
104.80953502984605
89.21850455545083
114.27348413446434
101.66627395538801
102.93041156142004
107.3605874960729
82.86365064404649
106.1875589066918
100.44769085768144
87.73798303487276
112.62213320766573
100.834904178448
94.28644360666037
91.51900722588752
85.5741438894125
119.53502984605717
100.93740182218032
109.9230285893811
99.38854853911405
96.57555765001571
101.27906063462142
112.45130380144516
104.17177191328935
100.92601319509897
98.68245366006911
100.51602262016965
94.28644360666037
90.8926327364122
93.

In [9]:
from scipy.stats import pearsonr
correlation = []

# chacking for each MONTH
month_path = slp_months[1] #0th in dex means january
# reading month data
month_data = pd.read_csv(month_path)
month_data.head()
# len(month_data)

,year,1,2,3,4,5,6,7,8,9,...,315,316,317,318,319,320,321,322,323,324
0,1948,-163.789145,10.608730,188.665968,201.223873,127.864248,32.278713,24.352798,16.966268,12.275152,...,7.510439,-0.419870,-10.222766,5.329311,6.772387,-31.013266,44.577556,271.336094,253.514316,270.482500
1,1949,-145.274314,-396.000828,-160.813463,170.886043,345.893728,206.242886,118.891189,34.684408,6.571417,...,2.001589,-3.169931,-34.453357,-50.756383,-42.205152,-56.292135,217.152446,487.232273,639.986117,890.509599
2,1950,-108.747946,-101.054661,-279.047167,-285.888493,-75.985850,91.758328,73.875747,-7.529032,-3.538690,...,-10.827330,-13.938852,-34.146533,-27.083470,51.104358,84.588724,9.822490,125.580296,386.674472,503.094255
3,1951,4.037088,34.271511,-52.911547,-255.556889,-254.734019,-83.716892,-48.382981,-90.506266,-19.968988,...,-11.054075,-9.903818,-17.248095,-7.619115,58.900744,79.077250,44.820659,125.627415,265.421908,269.103532
4,1952,152.176126,114.775173,-102.570177,-59.142765,-17.956553,-42.883823,-76.263169,-94.947306,-36.176752,...,-31.225889,-30.600595,-34.589282,-16.766148,21.897693,71.644205,177.624797,323.772373,258.057407,218.790361


In [14]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
combine = []
for i in range(12):
  # chacking for each MONTH
  month_path = slp_months[i] #0th in dex means january
  # reading month data
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:53]  #Here i am using anly data from 1948 to 2000
  for i in feature:
    combine.append(i)
x = np.array(combine)
x.shape

(636, 324)

In [15]:
# define encoder
def get_model_1(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  # encoder level 1
  e = Dense(n_inputs)(visible)
  # e = BatchNormalization()(e)
  e = ReLU()(e)
  

  # bottleneck
  # n_bottleneck = round(float(n_inputs) / 2.0)
  bottleneck = Dense(97)(e)
  # e = BatchNormalization()(bottleneck)

  # decoder level 1
  # d = Dense(n_inputs*2)(bottleneck)
  # d = BatchNormalization()(d)
  # d = LeakyReLU()(d)
  # output layer
  output = Dense(n_inputs, activation='linear')(bottleneck)
  # define autoencoder model
  model = Model(inputs=visible, outputs=output)
  # compile autoencoder model
  model.compile(optimizer='adam', loss='mse')
  # plot the autoencoder
  # plot_model(model, 'autoencoder_compress.png', show_shapes=True)
  return model

In [16]:
model_1 = get_model_1()
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_3 (Dense)             (None, 324)               105300    
                                                                 
 re_lu_1 (ReLU)              (None, 324)               0         
                                                                 
 dense_4 (Dense)             (None, 97)                31525     
                                                                 
 dense_5 (Dense)             (None, 324)               31752     
                                                                 
Total params: 168,577
Trainable params: 168,577
Non-trainable params: 0
_________________________________________________________________


In [17]:
# fit the autoencoder model to reconstruct input
history = model_1.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/800
40/40 - 0s - loss: 10779.0186 - val_loss: 6462.2471 - 399ms/epoch - 10ms/step
Epoch 2/800
40/40 - 0s - loss: 5480.7104 - val_loss: 4297.2378 - 66ms/epoch - 2ms/step
Epoch 3/800
40/40 - 0s - loss: 3782.2920 - val_loss: 3093.1953 - 67ms/epoch - 2ms/step
Epoch 4/800
40/40 - 0s - loss: 2857.8735 - val_loss: 2427.3049 - 66ms/epoch - 2ms/step
Epoch 5/800
40/40 - 0s - loss: 2312.8821 - val_loss: 2015.4062 - 68ms/epoch - 2ms/step
Epoch 6/800
40/40 - 0s - loss: 1949.5199 - val_loss: 1706.4534 - 72ms/epoch - 2ms/step
Epoch 7/800
40/40 - 0s - loss: 1682.2957 - val_loss: 1515.0272 - 67ms/epoch - 2ms/step
Epoch 8/800
40/40 - 0s - loss: 1484.6824 - val_loss: 1331.4739 - 70ms/epoch - 2ms/step
Epoch 9/800
40/40 - 0s - loss: 1340.3032 - val_loss: 1208.0149 - 74ms/epoch - 2ms/step
Epoch 10/800
40/40 - 0s - loss: 1224.6948 - val_loss: 1110.6838 - 84ms/epoch - 2ms/step
Epoch 11/800
40/40 - 0s - loss: 1127.2927 - val_loss: 1028.6899 - 74ms/epoch - 2ms/step
Epoch 12/800
40/40 - 0s - loss: 1049.1

In [18]:
# define encoder
def get_model_2(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  # encoder level 1
  e = Dense(n_inputs)(visible)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  e = Dense(97)(e)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  # bottleneck
  # n_bottleneck = round(float(n_inputs) / 2.0)
  bottleneck = Dense(29)(e)
  # e = BatchNormalization()(bottleneck)

  e = Dense(97)(bottleneck)
  e = ReLU()(e)
  # e = BatchNormalization()(e)

  # decoder level 1
  # d = Dense(n_inputs*2)(bottleneck)
  # d = BatchNormalization()(d)
  # d = LeakyReLU()(d)
  # output layer
  output = Dense(n_inputs, activation='linear')(e)
  # define autoencoder model
  model = Model(inputs=visible, outputs=output)
  # compile autoencoder model
  model.compile(optimizer='adam', loss='mse')
  # plot the autoencoder
  # plot_model(model, 'autoencoder_compress.png', show_shapes=True)
  return model

In [19]:
model_2 = get_model_2()
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_6 (Dense)             (None, 324)               105300    
                                                                 
 re_lu_2 (ReLU)              (None, 324)               0         
                                                                 
 dense_7 (Dense)             (None, 97)                31525     
                                                                 
 re_lu_3 (ReLU)              (None, 97)                0         
                                                                 
 dense_8 (Dense)             (None, 29)                2842      
                                                                 
 dense_9 (Dense)             (None, 97)                2910

In [20]:
weight_transfer_layer_model_2 = [1,2,3,8]
weight_transfer_from_layer_model_1 = [1,2,3,4]

for i in range(len(weight_transfer_layer_model_2)):
  model_2_index = weight_transfer_layer_model_2[i]
  model_1_index = weight_transfer_from_layer_model_1[i]
  model_2.layers[model_2_index] = model_1.layers[model_1_index]
  model_2.layers[model_2_index].trainable = False

In [21]:
# fit the autoencoder model to reconstruct input
history = model_2.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/800
40/40 - 0s - loss: 12222.0459 - val_loss: 8467.6055 - 386ms/epoch - 10ms/step
Epoch 2/800
40/40 - 0s - loss: 7692.3774 - val_loss: 6887.3105 - 69ms/epoch - 2ms/step
Epoch 3/800
40/40 - 0s - loss: 6454.8770 - val_loss: 5712.1489 - 70ms/epoch - 2ms/step
Epoch 4/800
40/40 - 0s - loss: 5414.0537 - val_loss: 4855.6377 - 72ms/epoch - 2ms/step
Epoch 5/800
40/40 - 0s - loss: 4609.4062 - val_loss: 4163.4883 - 69ms/epoch - 2ms/step
Epoch 6/800
40/40 - 0s - loss: 3985.4182 - val_loss: 3679.0085 - 69ms/epoch - 2ms/step
Epoch 7/800
40/40 - 0s - loss: 3572.6655 - val_loss: 3293.8984 - 69ms/epoch - 2ms/step
Epoch 8/800
40/40 - 0s - loss: 3201.8706 - val_loss: 2961.2703 - 71ms/epoch - 2ms/step
Epoch 9/800
40/40 - 0s - loss: 2946.7327 - val_loss: 2811.4126 - 70ms/epoch - 2ms/step
Epoch 10/800
40/40 - 0s - loss: 2731.0903 - val_loss: 2565.0342 - 70ms/epoch - 2ms/step
Epoch 11/800
40/40 - 0s - loss: 2567.8538 - val_loss: 2430.3030 - 70ms/epoch - 2ms/step
Epoch 12/800
40/40 - 0s - loss: 2420.4

In [22]:
# define encoder
def get_model_3(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  # encoder level 1
  e = Dense(n_inputs)(visible)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  e = Dense(97)(e)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  e = Dense(29)(e)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  # bottleneck
  # n_bottleneck = round(float(n_inputs) / 2.0)
  bottleneck = Dense(9)(e)
  # e = BatchNormalization()(bottleneck)

  e = Dense(29)(bottleneck)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  e = Dense(97)(e)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  # decoder level 1
  # d = Dense(n_inputs*2)(bottleneck)
  # d = BatchNormalization()(d)
  # d = LeakyReLU()(d)
  # output layer
  output = Dense(n_inputs, activation='linear')(e)
  # define autoencoder model
  model = Model(inputs=visible, outputs=output)
  # compile autoencoder model
  model.compile(optimizer='adam', loss='mse')
  # plot the autoencoder
  # plot_model(model, 'autoencoder_compress.png', show_shapes=True)
  return model

In [23]:
model_3 = get_model_3()
model_3.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_11 (Dense)            (None, 324)               105300    
                                                                 
 re_lu_5 (ReLU)              (None, 324)               0         
                                                                 
 dense_12 (Dense)            (None, 97)                31525     
                                                                 
 re_lu_6 (ReLU)              (None, 97)                0         
                                                                 
 dense_13 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_7 (ReLU)              (None, 29)                0   

In [24]:
# fit the autoencoder model to reconstruct input
history = model_3.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/1000
40/40 - 0s - loss: 13147.6631 - val_loss: 10346.4580 - 438ms/epoch - 11ms/step
Epoch 2/1000
40/40 - 0s - loss: 8856.4170 - val_loss: 7904.6987 - 71ms/epoch - 2ms/step
Epoch 3/1000
40/40 - 0s - loss: 7785.8442 - val_loss: 7520.9985 - 69ms/epoch - 2ms/step
Epoch 4/1000
40/40 - 0s - loss: 7360.2524 - val_loss: 7034.9585 - 73ms/epoch - 2ms/step
Epoch 5/1000
40/40 - 0s - loss: 6891.3379 - val_loss: 6556.5488 - 71ms/epoch - 2ms/step
Epoch 6/1000
40/40 - 0s - loss: 6462.5005 - val_loss: 6164.0415 - 70ms/epoch - 2ms/step
Epoch 7/1000
40/40 - 0s - loss: 6133.1621 - val_loss: 5926.2744 - 71ms/epoch - 2ms/step
Epoch 8/1000
40/40 - 0s - loss: 5837.1602 - val_loss: 5599.1665 - 71ms/epoch - 2ms/step
Epoch 9/1000
40/40 - 0s - loss: 5549.1685 - val_loss: 5357.2104 - 71ms/epoch - 2ms/step
Epoch 10/1000
40/40 - 0s - loss: 5336.6919 - val_loss: 5175.1128 - 77ms/epoch - 2ms/step
Epoch 11/1000
40/40 - 0s - loss: 5166.4185 - val_loss: 4966.5049 - 71ms/epoch - 2ms/step
Epoch 12/1000
40/40 - 0s -

In [25]:
model_final1 = Model(inputs=model_3.input, outputs=model_3.layers[1].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_3.input, outputs=model_3.layers[3].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_3.input, outputs=model_3.layers[5].output)
model_final3.trainable = False

model_final4 = Model(inputs=model_3.input, outputs=model_3.layers[7].output)
model_final4.trainable = False

models = [model_final1, model_final2, model_final3, model_final4]
# models = [model_final1, model_final4]

In [36]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[2:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
def predictor(month,model_final):
  # chacking for each MONTH
  month_path = slp_months[month] #0th in dex means january
  # reading month data
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:53]  #Here i am using anly data from 1948 to 2000
  pred = np.array(feature)
  print(pred)
  pred_m = model_final(pred)
  pred_f = pred_m.numpy()
  return pred_f

In [27]:
def get_top_pred(top,pred_info):
  k = 0
  for m in months:
    pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(0,53):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[0:53],score)
    cor_list.append(abs(corr))
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)

In [28]:
# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models):
  slp = predictor(months[0],models[0])
  print(slp.shape)
  pred = get_top_pred(top, slp)
  print(len(pred[0]))
  pred = np.array(pred)
  for mod in models[1:]:
    slp = predictor(months[0],mod)
    b = get_top_pred(top, slp)
    pred = np.concatenate((pred,b), axis=1)
  if(len(months)>1):
    for i in months[1:]:
      for mod in models[0:]:
        slp = predictor(i,mod)
        b = get_top_pred(top, slp)
        pred = np.concatenate((pred,b), axis=1)
  return pred

In [29]:
# pred = get_top_pred(4,pred_f)
from sklearn import preprocessing
months = [0,1,2]
pred = get_predictors(months,5,models)

ValueError: Input 0 of layer "model_4" is incompatible with the layer: expected shape=(None, 324), found shape=(73, 323)

In [ ]:
#This gave 0.76 for month feb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR

tree = DecisionTreeRegressor(max_depth=4)
from sklearn.ensemble import RandomForestRegressor
cor_all = []
reg_f = SVR(kernel = 'rbf')
for i in range(1,6):
  for j in range(40):
    reg = BaggingRegressor( n_estimators=i, random_state=j).fit(pred[:53], rain_fall_data[:53])
    regr = RandomForestRegressor(max_depth=i, random_state=j)
    reg_f.fit(pred[:53], rain_fall_data[:53])
    preds = reg_f.predict(pred[53:67])
    corr, _ = pearsonr(rain_fall_data[53:67],preds)
    cor_all.append(corr)
    print("depth ",i,"    ",corr)
print(min(cor_all),"   ",max(cor_all))

depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      

In [37]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  k = 0
  for m in months:
    pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  slp = predictor(months[0],models[0])
  # print(slp.shape)
  pred = get_top_pred(top, slp, w_start,w_end)
  # print(len(pred[0]))
  pred = np.array(pred)
  for mod in models[1:]:
    slp = predictor(months[0],mod)
    b = get_top_pred(top, slp,w_start,w_end)
    pred = np.concatenate((pred,b), axis=1)
  if(len(months)>1):
    for i in months[1:]:
      for mod in models[0:]:
        slp = predictor(i,mod)
        b = get_top_pred(top, slp, w_start,w_end)
        pred = np.concatenate((pred,b), axis=1)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      print(len(pred))
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [38]:
months_comb = [[4]]
top = 9
models = [model_final3 ]
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print(" Month combination = ",month,"    top = ",i,"   min and max",left,"   ",right)

[[-9.60277141e+01 -3.40342064e-01  8.12326677e+00 ... -7.32437426e+00
  -5.21531079e+01 -6.57146941e+01]
 [ 9.88139607e+01 -1.18701914e+02 -4.71935057e+01 ... -1.01359347e+02
  -7.44668286e+01 -5.22533293e+01]
 [ 9.96105539e+00  1.08338369e+02  5.10344606e+01 ...  1.09078763e+02
   2.41474456e+02  3.39467008e+02]
 ...
 [-6.89241375e+01  5.91380833e+00  3.32895876e+01 ... -2.16449985e+02
  -3.45451692e+02 -4.06605014e+02]
 [ 5.74530598e+01  2.42937522e+01  3.59381471e+01 ... -9.19543791e+01
  -1.89847627e+02 -1.87423129e+02]
 [-2.04032841e+02 -1.16579356e+02 -2.44869368e+00 ... -6.28537932e+01
  -1.90603303e+02  1.05589265e+01]]


ValueError: Input 0 of layer "model_6" is incompatible with the layer: expected shape=(None, 324), found shape=(53, 323)